In [3]:
import numpy as np
from scipy.optimize import fsolve, differential_evolution, minimize
from scipy.linalg import eig
import plotly.graph_objects as go
import matplotlib.pyplot as plt

In [4]:
def kappa(gammas):
    return np.sqrt(gammas[0] * gammas[3] / 4 / ALPHA / BETA)

def theta(gammas, Omega):
    return (Omega * gammas[3] / ALPHA / BETA / 2) ** (1/3)

def A_2_A_4_sys(vars, gammas, Omega):
    x, y = vars
    k = kappa(gammas)
    t = theta(gammas, Omega)
    
    c_6 = - k**4*x - k**2*y**2 + k**6 - x*y*k**2
    b_4 = y**2 + k**2 * x - k**4
    a_2 = k**2 - y
    
    term = k**2 * c_6**2 / t**6
    
    eq1 = b_4**2 + b_4*a_2*y - a_2**2*x*y - a_2*y**2*x - term
    eq2 = a_2*b_4*x + a_2**2*x*y - a_2*x**2*y - x**2*y**2 - term
    
    return [eq1, eq2]

def A_2_A_4(gammas, Omega, method='de', tol=1e-20, maxiter=500):
    try:
        denominator = 16 * ALPHA * BETA * gammas[3]
        if denominator == 0:
            return (np.nan, np.nan)
            
        initial_guess_val = np.sqrt(Omega - np.sqrt(gammas[1] * (gammas[4] * gammas[2] + gammas[0] * gammas[3])**2 / denominator))
        
        if not np.isfinite(initial_guess_val):
            return (np.nan, np.nan)
            
        lower_bound = max(0, Omega) 
        upper_bound = max(Omega, lower_bound + 1) 
        bounds = [(lower_bound, upper_bound), (lower_bound, upper_bound)]
        
        def objective(vars):
            eq1, eq2 = A_2_A_4_sys(vars, gammas, Omega)
            return eq1**2 + eq2**2
            
        if method == 'fsolve':
            solution = fsolve(lambda vars: A_2_A_4_sys(vars, gammas, Omega), 
                            [initial_guess_val, initial_guess_val], 
                            xtol=tol, 
                            maxfev=maxiter)
            
        elif method == 'de':
            result = differential_evolution(objective, bounds, tol=tol, maxiter=maxiter)
            solution = result.x
            
        elif method == 'hybrid':
            result_de = differential_evolution(objective, bounds, tol=tol, maxiter=maxiter//2)
            result_fsolve = fsolve(lambda vars: A_2_A_4_sys(vars, gammas, Omega), 
                                 result_de.x, 
                                 xtol=tol, 
                                 maxfev=maxiter//2)
            solution = result_fsolve
            
        x, y = solution

            
        return (np.sqrt(x), np.sqrt(y))
        
    except Exception as e:
        print(f"Ошибка при вычислении A_2_A_4: {str(e)}")
        return (np.nan, np.nan)

def calc_3rd_order_st_amps(gammas, Omega):
    A_2, A_4 = A_2_A_4(gammas, Omega)
    k = kappa(gammas)
    t = theta(gammas, Omega)
    c_6 = - k**4*A_2**2 - k**2*A_4**4 + k**6 - A_2**2*A_4**2*k**2
    b_4 = A_4**4 + k**2 * A_2**2 - k**4
    a_2 = k**2 - A_4**2

    A2_st = A_2                                               # phi_2 = 0
    A4_st = A_4                                               # phi_4 = 0
    A3_st = 1j*t**3 * A4_st * np.conj(A2_st) * a_2 / c_6      # phi_3 = pi/2
    A1_st = 1j*t**3 * b_4 / c_6                               # phi_1 = pi/2
    A5_st = 1j*t**3 * A4_st * A4_st * A2_st * A2_st / c_6     # phi_5 = pi/2
    rho1_st = 2j / gammas[3] * BETA * (A1_st * np.conj(A2_st) + A3_st * np.conj(A4_st))
    rho2_st = 2j / gammas[3] * BETA * (A2_st * np.conj(A3_st) + A4_st * np.conj(A5_st))

    return (A1_st, A2_st, A3_st, A4_st, A5_st, rho1_st, rho2_st)

def build_matrix(gammas, Omega):
    A1_st, A2_st, A3_st, A4_st, A5_st, rho1_st, rho2_st = calc_3rd_order_st_amps(gammas, Omega)
    M = np.zeros((14, 14), dtype=complex)
    
    M[0,0] = -gammas[0] / 2
    M[0,1] = 1j*ALPHA*rho1_st
    M[0,5] = 1j*ALPHA*A2_st

    M[1,0] = 1j*ALPHA*np.conj(rho1_st)
    M[1,1] = -gammas[1] / 2
    M[1,2] = 1j*ALPHA*rho2_st
    M[1,6] = 1j*ALPHA*A3_st
    M[1,12] = 1j*ALPHA*A1_st

    M[2,1] = 1j*ALPHA*np.conj(rho2_st)
    M[2,2] = -gammas[2] / 2
    M[2,3] = 1j*ALPHA*rho1_st
    M[2,5] = 1j*ALPHA*A4_st
    M[2,13] = 1j*ALPHA*A2_st

    M[3,2] = 1j*ALPHA*np.conj(rho1_st)
    M[3,3] = -gammas[2] / 2
    M[3,4] = 1j*ALPHA*rho2_st
    M[3,6] = 1j*ALPHA*A5_st
    M[3,12] = 1j*ALPHA*A3_st

    M[4,3] = 1j*ALPHA*np.conj(rho2_st)
    M[4,4] = -gammas[2] / 2
    M[4,13] = 1j*ALPHA*A4_st

    M[5,0] = 1j*BETA*np.conj(A2_st)
    M[5,2] = 1j*BETA*np.conj(A4_st)
    M[5,5] = -gammas[3] / 2
    M[5,8] = 1j*BETA*A1_st
    M[5,10] = 1j*BETA*A3_st

    M[6,1] = 1j*BETA*np.conj(A3_st)
    M[6,3] = 1j*BETA*np.conj(A5_st)
    M[6,6] = -gammas[4] / 2
    M[6,9] = 1j*BETA*A2_st
    M[6,11] = 1j*BETA*A4_st

    M[7,7] = -np.conj(gammas[0]) / 2
    M[7,8] = -1j*ALPHA*np.conj(rho1_st)
    M[7,12] = -1j*ALPHA*np.conj(A2_st)

    M[8,7] = -1j*ALPHA*rho1_st
    M[8,8] = -np.conj(gammas[1]) / 2
    M[8,9] = -1j*ALPHA*np.conj(rho2_st)
    M[8,13] = -1j*ALPHA*np.conj(A3_st)
    M[8,5] = -1j*ALPHA*np.conj(A1_st)

    M[9,8] = -1j*ALPHA*rho2_st
    M[9,9] = -np.conj(gammas[2]) / 2
    M[9,10] = -1j*ALPHA*np.conj(rho1_st)
    M[9,12] = -1j*ALPHA*np.conj(A4_st)
    M[9,6] = -1j*ALPHA*np.conj(A2_st)

    M[10,9] = -1j*ALPHA*rho1_st
    M[10,10] = -np.conj(gammas[2]) / 2
    M[10,11] = -1j*ALPHA*np.conj(rho2_st)
    M[10,13] = -1j*ALPHA*np.conj(A5_st)
    M[10,5] = -1j*ALPHA*np.conj(A3_st)

    M[11,10] = -1j*ALPHA*rho2_st
    M[11,11] = -np.conj(gammas[2]) / 2
    M[11,6] = -1j*ALPHA*np.conj(A4_st)

    M[12,7] = -1j*BETA*A2_st
    M[12,9] = -1j*BETA*A4_st
    M[12,12] = -np.conj(gammas[3]) / 2
    M[12,1] = -1j*BETA*np.conj(A1_st)
    M[12,3] = -1j*BETA*np.conj(A3_st)

    M[13,8] = -1j*BETA*A3_st
    M[13,10] = -1j*BETA*A5_st
    M[13,13] = -np.conj(gammas[4]) / 2
    M[13,2] = -1j*BETA*np.conj(A2_st)
    M[13,4] = -1j*BETA*np.conj(A4_st)

    return M, (A1_st, A2_st, A3_st, A4_st, A5_st, rho1_st, rho2_st)

In [6]:
gammas = np.array([10, 10, 10, 1, 1])
Omega_values = np.linspace(0, 60, 50)
ALPHA = 1
BETA = 1

stability = []
amps = []

for Omega in Omega_values:
    M, amp= build_matrix(gammas, Omega)
    amps.append(amp)
    try:
        eigenvalues = eig(M)[0]
        max_real = np.max(np.real(eigenvalues))
        stability.append(max_real)
    except:
        stability.append(np.nan)

amps = np.array(amps)

fig = go.Figure()


for i in range(5):
    fig.add_trace(go.Scatter(
        x=Omega_values,
        y=np.abs(amps[:, i]),
        name=f'A{i+1}',
        mode='lines',
        line=dict(width=2),
    ))

for i in range(2):
    fig.add_trace(go.Scatter(
        x=Omega_values,
        y=np.abs(amps[:, 5+i]),
        name=f'rho{i+1}',
        mode='lines',
        line=dict(width=2),
    ))

fig.update_layout(
    title='Зависимость амплитуд от накачки Omega',
    xaxis_title='Накачка Omega',
    yaxis_title='Амплитуда',
    legend_title='Амплитуды',
    hovermode='x unified', 
    template='plotly_white', 
)
# fig.add_hline(y=kappa(gammas), line_dash="dash", line_color="green")
fig.update_xaxes(showspikes=True, spikecolor='gray', spikesnap='cursor', spikemode='across')
fig.update_yaxes(showspikes=True, spikecolor='gray', spikesnap='cursor', spikemode='across')
fig.update_layout(
    xaxis_title='Ω',
    height=600,
    width=800
)
fig.show()
plt.show()

C:\Users\user\AppData\Local\Temp\ipykernel_5336\469090.py:29: RuntimeWarning:

invalid value encountered in sqrt



In [ ]:
gammas = np.array([1e5, 1e5, 1e5, 1, 1])
Omega_values = np.linspace(0, 2e8, 1000)
ALPHA = 1
BETA = 1

stability = []
amps = []

for Omega in Omega_values:
    M, amp = build_matrix(gammas, Omega)
    amps.append(amp)
    try:
        eigenvalues = eig(M)[0]
        max_real = np.max(np.real(eigenvalues))
        stability.append(max_real)
    except:
        stability.append(np.nan)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=Omega_values, y=stability,
    mode='lines',
    name='Zero Order',
    line=dict(color='blue')
))

fig.update_layout(
    xaxis_title='Ω',
    height=600,
    width=800
)

fig.show()